In [4]:
import numpy as np
import random

def adjust_schedule(X, N, D, A, B, dayoff):
    # Điều chỉnh lịch để mỗi ca có từ A đến B nhân viên
    for d in range(1, D + 1):
        for shift in range(1, 5):  # Các ca 1, 2, 3, 4
            count = np.sum(X[:, d] == shift)
            while count < A:
                available = [i for i in range(1, N + 1) if X[i][d] == 0 and dayoff[i][d] == 0 and (d == 1 or X[i][d - 1] != 4)]
                if not available:
                    break
                i = random.choice(available)
                X[i][d] = shift
                if shift == 4 and d < D:
                    X[i][d + 1] = 0  # Nghỉ sau ca đêm
                count += 1
            while count > A:
                assigned = [i for i in range(1, N + 1) if X[i][d] == shift]
                if not assigned:
                    break
                i = random.choice(assigned)
                X[i][d] = 0
                count -= 1
    # Sửa lỗi ca đêm (ngày sau ca đêm phải nghỉ)
    for i in range(1, N + 1):
        for d in range(1, D):
            if X[i][d] == 4 and X[i][d + 1] != 0:
                X[i][d + 1] = 0

    # Đảm bảo các ca không bị thiếu
    for d in range(1, D + 1):
        for shift in range(1, 5):  # Các ca 1, 2, 3, 4
            count = np.sum(X[:, d] == shift)
            while count < A:
                available = [i for i in range(1, N + 1) if X[i][d] == 0 and dayoff[i][d] == 0 and (d == 1 or X[i][d - 1] != 4)]
                if not available:
                    break
                i = random.choice(available)
                X[i][d] = shift
                if shift == 4 and d < D:
                    X[i][d + 1] = 0  # Nghỉ sau ca đêm
                count += 1
            while count > B:
                assigned = [i for i in range(1, N + 1) if X[i][d] == shift]
                if not assigned:
                    break
                i = random.choice(assigned)
                X[i][d] = 0
                count -= 1
    return X

class AntColony:
    def __init__(self, N, D, A, B, dayoff, num_ants=15, max_iterations=30, alpha=1, beta=2, rho=0.1, Q=1):
        self.N = N  # Số nhân viên
        self.D = D  # Số ngày
        self.A = A  # Số nhân viên tối thiểu mỗi ca
        self.B = B  # Số nhân viên tối đa mỗi ca
        self.dayoff = dayoff  # Ma trận ngày nghỉ
        self.num_ants = num_ants
        self.max_iterations = max_iterations
        self.alpha = alpha # Trọng số pheromone
        self.beta = beta # Trọng số heuristic
        self.rho = rho # Hệ số bay hơi pheromone
        self.Q = Q # Hệ số pheromone
        self.pheromone = self.initialize_pheromone()

    def initialize_pheromone(self):
        pheromone = {}
        for i in range(1, self.N + 1):
            for d in range(1, self.D + 1):
                for s in range(5):  # 0: nghỉ, 1-4: các ca
                    pheromone[(i, d, s)] = 0.1
        return pheromone

    def heuristic(self, i, d, s, X):
        # Giá trị heuristic
        if self.dayoff[i][d] == 1 and s != 0:
            return 0
        if d > 1 and X[i][d - 1] == 4 and s != 0:
            return 0
        if s == 4:
            return 0.1  # Ưu tiên ít ca đêm
        return 1

    def construct_solution(self):
        X = np.zeros((self.N + 1, self.D + 1))
        for i in range(1, self.N + 1):
            for d in range(1, self.D + 1):
                if self.dayoff[i][d] == 1:
                    X[i][d] = 0
                    continue
                allowed_shifts = [0] if (d > 1 and X[i][d - 1] == 4) else range(5)
                probs = []
                for s in allowed_shifts:
                    tau = self.pheromone[(i, d, s)]
                    eta = self.heuristic(i, d, s, X)
                    prob = (tau ** self.alpha) * (eta ** self.beta)
                    probs.append(prob)
                probs = np.array(probs) / np.sum(probs)
                s = np.random.choice(allowed_shifts, p=probs)
                X[i][d] = s
        X = adjust_schedule(X, self.N, self.D, self.A, self.B, self.dayoff)
        return X

    def evaluate_fitness(self, X):
        # Tối thiểu hóa số ca đêm tối đa của một nhân viên
        max_night = 0
        for i in range(1, self.N + 1):
            night_count = np.sum(X[i] == 4)
            max_night = max(max_night, night_count)
        return -max_night

    def update_pheromone(self, solutions, fitness_scores):
        # Bay hơi pheromone
        for key in self.pheromone:
            self.pheromone[key] *= (1 - self.rho)
        # Cộng thêm pheromone từ các lời giải
        for k in range(self.num_ants):
            X = solutions[k]
            fitness = fitness_scores[k]
            cost = -fitness  # Chuyển fitness thành cost
            delta_tau = self.Q / (cost + 1e-10)  # Tránh chia cho 0
            for i in range(1, self.N + 1):
                for d in range(1, self.D + 1):
                    s = int(X[i][d])
                    self.pheromone[(i, d, s)] += delta_tau

    def run(self):
        best_solution = None
        best_fitness = float('-inf')
        for iteration in range(self.max_iterations):
            print(f"Iteration {iteration + 1}/{self.max_iterations}")
            if iteration > 0:
                # In gia tri toi uu hien tai
                print(f"Best fitness: {best_fitness}")
            solutions = []
            fitness_scores = []
            for _ in range(self.num_ants):
                X = self.construct_solution()
                fitness = self.evaluate_fitness(X)
                solutions.append(X)
                fitness_scores.append(fitness)
                if fitness > best_fitness:
                    best_solution = X
                    best_fitness = fitness
            self.update_pheromone(solutions, fitness_scores)
        return best_solution

def read_input_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    N, D, A, B = map(int, lines[0].split())
    dayoff = np.zeros((N+1, D+1))
    for i in range(1, N+1):
        numbers = list(map(int, lines[i].split()))
        for j in range(0, len(numbers) - 1):
            dayoff[i][numbers[j]] = 1
    return N, D, A, B, dayoff

# N, D, A, B, dayoff = read_input_from_file('Testcase/tc2.txt')

# ant_colony = AntColony(N, D, A, B, dayoff)
# best_solution = ant_colony.run()
# print("Lịch làm việc tốt nhất:")
# print(best_solution[1:, 1:])
import time

filename = 'Testcase/tc'
output = 'Output/output'
list_test = [1]
for tc in list_test:
    start_time = time.time()
    input_file = f"{filename}{tc}.txt"
    output_file = f"{output}{tc}.txt"

    N, D, A, B, dayoff = read_input_from_file(input_file)
    ant_colony = AntColony(N, D, A, B, dayoff)
    best_solution = ant_colony.run()
    end_time = time.time()
    with open(output_file, 'w') as file:
        for i in range(1, N + 1):
            file.write(' '.join([str(int(best_solution[i][d])) for d in range(1, D + 1)]) + '\n')
        end_time = time.time()
        file.write("Execution time: {:.2f} seconds\n".format(end_time - start_time))
    print(f"Test case {tc} completed in {end_time - start_time:.2f} seconds.")



Iteration 1/30
Iteration 2/30
Best fitness: -1
Iteration 3/30
Best fitness: -1
Iteration 4/30
Best fitness: -1
Iteration 5/30
Best fitness: -1
Iteration 6/30
Best fitness: -1
Iteration 7/30
Best fitness: -1
Iteration 8/30
Best fitness: -1
Iteration 9/30
Best fitness: -1
Iteration 10/30
Best fitness: -1
Iteration 11/30
Best fitness: -1
Iteration 12/30
Best fitness: -1
Iteration 13/30
Best fitness: -1
Iteration 14/30
Best fitness: -1
Iteration 15/30
Best fitness: -1
Iteration 16/30
Best fitness: -1
Iteration 17/30
Best fitness: -1
Iteration 18/30
Best fitness: -1
Iteration 19/30
Best fitness: -1
Iteration 20/30
Best fitness: -1
Iteration 21/30
Best fitness: -1
Iteration 22/30
Best fitness: -1
Iteration 23/30
Best fitness: -1
Iteration 24/30
Best fitness: -1
Iteration 25/30
Best fitness: -1
Iteration 26/30
Best fitness: -1
Iteration 27/30
Best fitness: -1
Iteration 28/30
Best fitness: -1
Iteration 29/30
Best fitness: -1
Iteration 30/30
Best fitness: -1
Test case 1 completed in 0.87 second

In [5]:
for i in range(1, N + 1):
    for d in range(1, D + 1):
        print(int(best_solution[i][d]), end=' ')
    print()
print("Số ca đêm tối đa:", -ant_colony.evaluate_fitness(best_solution))

2 4 0 3 0 0 
0 0 2 0 4 0 
0 2 3 4 0 0 
3 0 1 0 0 0 
0 0 0 1 0 0 
1 3 0 0 0 4 
0 0 4 0 1 2 
0 1 0 0 2 0 
0 0 0 2 0 3 
4 0 0 0 3 1 
Số ca đêm tối đa: 1


In [6]:
#check xem co ca nao bi vi pham khong
ngay_vi_pham = []
for i in range(1, N + 1):
    for d in range(1, D):
        if best_solution[i][d] == 4 and best_solution[i][d + 1] != 0:
            print(f"Nhân viên {i} vi phạm quy tắc ca đêm tại ngày {d}")

for d in range(1, D + 1):
    for shift in range(1, 5):  # Các ca 1, 2, 3, 4
        count = np.sum(best_solution[:, d] == shift)
        if count < A or count > B:
            print(f"Vi phạm quy tắc số lượng nhân viên cho ca {shift} tại ngày {d}")
            print (f"Số lượng nhân viên: {count}, yêu cầu: {A}-{B}")
            ngay_vi_pham.append(d)

for i in range(1, N + 1):
    for d in range(1, D + 1):
        if best_solution[i][d] != 0 and dayoff[i][d] == 1:
            print(f"Nhân viên {i} vi phạm quy tắc ngày nghỉ tại ngày {d}")

for d in ngay_vi_pham:
    print(f"Số nhân viên nghỉ tại ngày {d}:")
    print(np.sum(best_solution[:, d] == 0))